In [ ]:
# # no need for this since it has nothing outside classes and functions
# print(__name__)
# if __name__ == "__main__" and hasattr(__builtins__,'__IPYTHON__') and ('google.colab' in str(get_ipython())):
#     from google.colab import drive
#     drive.mount('/content/drive')
#     %cd /content/drive/MyDrive/PressureReliefWorkArea/SummerWork/
#     !ls

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
import numpy as np

%run -n HelperFunctions.ipynb
# import ipynb
# from ipynb.fs.full.HelperFunctions import *

This code loads your CSV file, splits the data into a training set and a test set, and creates a DataLoader for each. The DataLoader can be used to iterate through the data in batches, which is useful for training a neural network.

You can replace 'yourfile.csv' with the path to your actual file. Also, note that this assumes your CSV file doesn't have a header. If it does, you might need to skip the first row.

In [ ]:
class JFSKAccelDataset(Dataset):
    def __init__(self, data, labels, sequence_length=10):
        self.data = [data[i:i+sequence_length] for i in range(len(data) - sequence_length + 1)]
        self.labels = labels[(int)(sequence_length/2) - 1 : len(data) - (sequence_length - (int)(sequence_length/2))]
        # change to get the majority
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx].transpose(0, 1), self.labels[idx]  # Transposing the sequence and channel dimensions

In [ ]:
class JFLoader:
    def __init__(self, file_path, sequence_length = 10, repress_stationary = True):
        self.sequence_length = sequence_length

        # Gather file info
        # self.file_directory, self.beginning_descriptors, self.file_name, self.ending_descriptors, self.file_extension, self.specifier_values = SKFileNameHandler.read_data_file_name(file_path)
        self.file_directory, _, _, _, file_extension, self.specifier_values = SKFileNameHandler.read_data_file_name(file_path)
        input_num = SKDescriptors.NUM_OF_INPUTS_PER_TYPE[self.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]]

        # Load CSV file
        # WORKAREA_PATH = './'
        # class_num = 8 ## Although annoying, requiring this be manually adjusted
        #     ## to open the file also means that a later instance of this number will be correct
        # prev input: WORKAREA_PATH + f'Data/COMBINED_Type5-WithClassNum{class_num}-Freq10-Labeled_Motion-sessions_23-24_Fall.csv'

        match file_extension:
            case ".csv":
                dataframe = pd.read_csv(file_path)
            case _:
                raise NotImplementedError(f"JFLoader is not equipped to open {file_extension} files.")
        dataframe.columns = pd.Index(np.arange(len(dataframe.columns)), dtype='int64')
        # code test file: Data/Week 1/Left then Right/Processed/Type3-Freq10-Labeled_Motion-sessions_2023-08-26_17-25-54.csv
        # classifier training file: Data/COMBINED_Type3-Freq10-Labeled_Motion-sessions_23-24_Fall.csv


        if repress_stationary:
            # NOTE: the logic here only works for one-hot vectors
            if not SKDescriptors.NUM_OF_OUTPUTS_PER_TYPE[self.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]] == 1:
                raise NotImplementedError(f"JFLoader with repress_stationary=True may not be equipped to fairly sample stationary data for classification types with outputs that are not one-hot vectors.")
            # if "Stationary" is one of the classes, it may overpower the others,
                # so we will want to remove a portion of these to even the class counts
            # we first test if this is one of the classes
            # class_index = -1
            # temp = 0
            # for c in SKDescriptors.LIST_OF_CLASSES_AND_TAGS_PER_TYPE[self.specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]]:
            #     print(c)
            #     if c == "Stationary":
            #         class_index = temp
            #         break
            #     temp += 1

            # my population sample randomizer from STAT 2113
            # for i in range(SAMPLE_SIZE):
            #     choice_index = random.randint(1, unchosen_len) - 1
            #     choice = unchosen[choice_index]
            #     unchosen.pop(choice_index)
            #     unchosen_len -= 1
            #     choices_dict[choice] = choices_dict.setdefault(choice, 0)
            #     choices_dict[choice] += 1
            # choices_list = sorted(choices_dict)

            # we should not iterate over a dataframe
            # if class_index != -1:
            # check the counts of every class
            # this first removes the input columns;
                # then it removes the Stationary column;
                # then it counts each remaining columns' counts of '1' (everything inside .apply());
                    # inside .apply(), we count how many of each number we have;
                    # then we change it to a zip, then a dict;
                    # then we take the count from key 1,
                    # and if it doesn't have a 1 key, we return 0;
                # then it converts it to a numpy array;
                # then it takes the mean of the counts;
                # then it turns this into an int

            # use this if we want the mean, but take into account the fact that some values may overpower the others
            # avg_count = int(dataframe[dataframe.columns[3 : len(dataframe.columns) - 2]].apply(class_counter).to_numpy().mean())
            # # class_counts = {}
            # # for i in dataframe.columns[input_num : ]:
            # #     if i == SKDescriptors.STATIONARY_CLASS:
            # #         stationary_rows = dataframe[dataframe[i] == 1]
            # #         continue
            # #     class_counts[i] = len(dataframe[dataframe[i] == 1])
            # stationary_rows = dataframe[dataframe[len(dataframe.columns) - 2] == 1]
            # other_rows = dataframe[dataframe[len(dataframe.columns) - 1] == 1]
            # print()
            # print(avg_count)
            # print("statlen: " + str(len(stationary_rows)))
            # print("otherlen: " + str(len(other_rows)))
            # # this is the line doing the actual randomization
            # sample_rows = stationary_rows.sample(avg_count, random_state=42)
            # print(len(sample_rows))
            # dataframe = dataframe.drop(stationary_rows.drop(sample_rows.index).index)
            # print(len(dataframe))
            # # this is the line doing the actual randomization
            # sample_rows = other_rows.sample(avg_count, random_state=42)
            # print(len(sample_rows))
            # dataframe = dataframe.drop(other_rows.drop(sample_rows.index).index)
            # print(len(dataframe))


            # want min instead?
            # avg_count = int(dataframe[dataframe.columns[input_num : ]].apply(JFLoader.count_instances).to_numpy().min())

            # median
            # avg_count = int(np.median(dataframe[dataframe.columns[3 : ]].apply(lambda x: skcounter(x)).to_numpy()))

            # for i in dataframe.columns[input_num : ]:
            #     class_i_rows = dataframe[dataframe[i] == 1]
            #     # this is the line doing the actual randomization
            #     sample_rows = class_i_rows.sample(min(len(class_i_rows), avg_count), random_state=42)
            #     # print(len(sample_rows))
            #     dataframe = dataframe.drop(class_i_rows.drop(sample_rows.index).index)
            #     # print(len(dataframe))



            # avg_count = int(dataframe[dataframe.columns[input_num : ]].drop(SKDescriptors.STATIONARY_CLASS, axis=1).apply(lambda x: dict(zip(np.unique(x, return_counts = True))).get(1, 0)).to_numpy().mean())
            # # class_counts = {}
            # # for i in dataframe.columns[input_num : ]:
            # #     if i == SKDescriptors.STATIONARY_CLASS:
            # #         stationary_rows = dataframe[dataframe[i] == 1]
            # #         continue
            # #     class_counts[i] = len(dataframe[dataframe[i] == 1])
            # stationary_rows = dataframe[dataframe[SKDescriptors.STATIONARY_CLASS] == 1]
            # # this is the line doing the actual randomization
            # sample_rows = stationary_rows.sample(avg_count, random_state=42)
            # dataframe = dataframe.drop(stationary_rows.drop(sample_rows.index).index)






        # Get data and labels from dataframe
        data = dataframe.iloc[:, :input_num].values  # x, y, z data
        labels = dataframe.iloc[:, input_num:].values  # labels

        # sequence_length = 10

        # Split data into training and test sets
        data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        # Convert data to tensors
        data_train = torch.tensor(data_train, dtype=torch.float32)
        data_test = torch.tensor(data_test, dtype=torch.float32)

        # Convert labels to tensors and get max index (assuming one-hot encoding)
        labels_train = torch.argmax(torch.tensor(labels_train, dtype=torch.float32), dim=1)
        labels_test = torch.argmax(torch.tensor(labels_test, dtype=torch.float32), dim=1)

        # Create data loaders
        train_dataset = JFSKAccelDataset(data_train, labels_train, self.sequence_length)
        test_dataset = JFSKAccelDataset(data_test, labels_test, self.sequence_length)

        self.train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        self.test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


    def count_instances(series):
        ccarr = np.unique(series, return_counts = True)
        return dict(zip(ccarr[0], ccarr[1])).get(1, 0)
        # print(skarr)
        # find_index = np.where(skarr[0], True, False)
        # for i in range(len(find_index)):
        #     if find_index[i]: ret_valsk = skarr[1][i]
        # print(ret_dict)
        # ret_valsk = ret_dict.get(1, 0)
        # print(ret_valsk)
        # # print(dataframe)
        # return ret_valsk


In [ ]:
class SKInputConverter:
    def __init__(self, dataframe, classtype, bufferpref = "Inner"):
        self.dataframe = dataframe
        SKDescriptors.validate_class_type(classtype)
        self.classtype = classtype
        self.bufferpref = bufferpref
        self.inputnum = SKDescriptors.NUM_OF_INPUTS_PER_TYPE[classtype]

    def result(self):
        return self.dataframe, self.inputnum

    def diff(self):
        pass
        return self

    def remove_outliers(self, rem_type = None, rem_func = None, args = None):
        pass
        return self

    def normalize(self, norm_type = None, norm_func = None, args = None):
        pass
        return self

    def combine(self, comb_type = None, comb_func = None, args = None):
        if comb_type != None or comb_func != None or args != None:
            raise NotImplementedError(f"SKInputConverter.combine() has no implemented parameters")
        self.dataframe = self.dataframe.iloc[:,:self.inputnum].apply(np.linalg.norm).join(self.dataframe.iloc[:,self.inputnum:])
        self.dataframe.columns = pd.Index(np.arange(len(self.dataframe.columns) - self.inputnum + 1))
        self.inputnum = 1
        return self